In [17]:
# load the data as `data` and lines as `lines` variable
%run helper.py {11}

In [1]:
import numpy as np

data="""Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1"""

lines = data.split("\n")

In [18]:
monkey = dict()
dividers = []
for l in lines:
    l = l.strip()
    if l.startswith("Monkey"):
        _, n = l.split()
        n = int(n[:-1])
        monkey[n] = dict()
        monkey[n]["items"] = []
    elif l.startswith("Starting items"):
        monkey[n]["items"] = [int(i) for i in l[16:].split(", ")]
    elif l.startswith("Operation"):
        monkey[n]["inspection_counter"] = 0
        
        def func_inspect(old, monkey, n=n, l=l):
            new = eval(l[11:].split(" = ")[1])
            monkey[n]["items"].remove(old)
            monkey[n]["inspection_counter"] += 1
            return new, monkey
        monkey[n]["inspect"] = func_inspect
        
    elif l.startswith("Test"):
        divider = int(l.split()[-1])
        dividers.append(divider)
    elif l.startswith("If true"):
        target_true = int(l.split()[-1])
    elif l.startswith("If false"):
        target_false = int(l.split()[-1])

        def throw_item(item, monkey, n=n, d=divider,
                      tt=target_true, tf=target_false):
            item = item // 3
            # print("monkey ", n, "baca ", tt, " ili ", tf)
            if item%d == 0:
                monkey[tt]["items"].append(item)
                # print("Sending item ", item, " to monkey ", tt)
            else:
                monkey[tf]["items"].append(item)
                # print("Sending item ", item, " to monkey ", tf)
            return monkey
        monkey[n]["throw"] = throw_item


In [19]:
from copy import copy, deepcopy
from pprint import pprint

for it in range(20):
    for n in range(len(monkey)):
        for item in deepcopy(monkey[n]["items"]):
            new_item, monkey = monkey[n]["inspect"](item, monkey)
            monkey = monkey[n]["throw"](new_item, monkey)

# import numpy as np
np.prod(np.partition([m["inspection_counter"] for m in monkey.values()], -2)[-2:])
    

56595

In [20]:
uni = np.prod(dividers)

In [21]:
dividers

[17, 13, 19, 7, 11, 3, 2, 5]

In [9]:
dd = 321
glanc = [0, 0]
for ii in range(30000):
    k = ii**3
    novi = [k//dd, k%dd]
    if novi[0] > dd:
        glanc = [novi[0]//dd, (novi[0]-dd)*dd+novi[1]]
        print(f"{novi[0]}*{dd} + {novi[1]}, {glanc[0]}*{dd**2} = {glanc[0]*dd**2} + {glanc[1]}, a treba {k//dd**2}*{dd**2} + {k%dd**2}")
        novi = copy(glanc)
        dd = dd**2

323*321 + 140, 1*103041 = 103041 + 782, a treba 1*103041 + 782
103055*103041 + 96137, 1*10617447681 = 10617447681 + 1538711, a treba 1*10617447681 + 1538711


In [22]:
monkey = dict()
dev = 0
for l in lines:
    l = l.strip()
    if l.startswith("Monkey"):
        _, n = l.split()
        n = int(n[:-1])
        monkey[n] = dict()
        monkey[n]["items"] = []
    elif l.startswith("Starting items"):
        monkey[n]["items"] = [int(i) for i in l[16:].split(", ")]
    elif l.startswith("Operation"):
        monkey[n]["inspection_counter"] = 0
        ##################################################################
        def func_inspect(old, monkey, n=n, l=l, uni=uni):

            assert old >= 0, f"monkey {n}, {old_tuple}"
            new = eval(l[11:].split(" = ")[1])
            new = new % uni
            monkey[n]["items"].remove(old)
            monkey[n]["inspection_counter"] += 1
            return new, monkey
        ###################################################################
        monkey[n]["inspect"] = func_inspect
    elif l.startswith("Test"):
        divider = int(l.split()[-1])
    elif l.startswith("If true"):
        target_true = int(l.split()[-1])
    elif l.startswith("If false"):
        target_false = int(l.split()[-1])
        ###################################################################
        def throw_item(item, monkey, n=n, d=divider,
                      tt=target_true, tf=target_false):
            # item = item // divider + item%divider
            # print("monkey ", n, "baca ", tt, " ili ", tf)
            if item%d == 0:
                monkey[tt]["items"].append(item)
                # print("Sending item ", item, " to monkey ", tt)
            else:
                monkey[tf]["items"].append(item)
                # print("Sending item ", item, " to monkey ", tf)
            return monkey
        ##################################################################
        monkey[n]["throw"] = throw_item


In [23]:
from copy import copy, deepcopy

for it in range(10000):
    for n in range(len(monkey)):
        for item in deepcopy(monkey[n]["items"]):
            new_item, monkey = monkey[n]["inspect"](item, monkey)
            # if new_item[0] > 0 and np.log2(new_item[0]) > 40:
            #     print(new_item)
            monkey = monkey[n]["throw"](new_item, monkey)
    if it in [0, 19, 999, 1999, 2999, 3999, 4999, 5999, 6999, 7999, 8999, 9999]:
        print(f"\n== After round {it+1} ==")
        for nn in range(len(monkey)):
            print(f"Monkey {nn+1} inspected items {monkey[nn]['inspection_counter']:.>8d} times.")


most_actives = np.partition([m["inspection_counter"] for m in monkey.values()], -2)[-2:]
print(f"\n\nAfter {it+1} rounds, the two most active monkeys inspected items "
      f"{most_actives[0]} and {most_actives[1]} times.\n"
      f"Multiplying these together, the level of monkey buisness in this "
      f"situation is now {np.prod(most_actives)}")


== After round 1 ==
Monkey 1 inspected items .......7 times.
Monkey 2 inspected items .......6 times.
Monkey 3 inspected items ......10 times.
Monkey 4 inspected items .......5 times.
Monkey 5 inspected items .......6 times.
Monkey 6 inspected items ......18 times.
Monkey 7 inspected items .......9 times.
Monkey 8 inspected items .......5 times.

== After round 20 ==
Monkey 1 inspected items ......79 times.
Monkey 2 inspected items .....197 times.
Monkey 3 inspected items .....192 times.
Monkey 4 inspected items .....250 times.
Monkey 5 inspected items .....195 times.
Monkey 6 inspected items .....246 times.
Monkey 7 inspected items .....104 times.
Monkey 8 inspected items ......67 times.

== After round 1000 ==
Monkey 1 inspected items ....2269 times.
Monkey 2 inspected items ...10509 times.
Monkey 3 inspected items ...11184 times.
Monkey 4 inspected items ...12735 times.
Monkey 5 inspected items ...10498 times.
Monkey 6 inspected items ...12310 times.
Monkey 7 inspected items ....57